# RSA
Yesterday I started an attempt to implement [RSA](https://en.wikipedia.org/wiki/RSA_%28cryptosystem%29) by generating large prime numbers. Let’s continue in this direction and implement the encryption mechanism.

But first, we are going to need more theory. Again, don’t worry if you don’t feel comfortable with math. It’s there just to show why RSA works.

> disclaimer: do not consider my code to be secure; do not consider any cryptography coming from non-experts to be secure; you should never implement any kind of cryptography on your own nor should you interfere with your security in any way; this series is just for fun and as such should be taken

Assume we already have two sufficiently large primes $p$ and $q$. Use them to compute modulus $n$ and value of its [totient function](https://en.wikipedia.org/wiki/Euler%27s_totient_function) $\varphi(n)$.

$$prime numbers: p,q$$
$$modulus: n = pq$$
$$\varphi(n)=(p-1)(q-1)=n-p-q+1$$

Euler’s theorem states, if $a$ and $n$ are relatively prime, the congruence below always holds.

$$a^{\varphi(n)}\equiv1(modn)$$

Next we choose value of $e$. It is usually fixed and value 65537 is often used.

$$e=2^{16}+1$$
$$d\equiv e^{-1}(mod \varphi(n))$$

The final step is to find $d$ that is a multiplicative inverse of $e$ modulo $\varphi(n)$. Do we need another algorithm? Luckily, we can use extended euclidean from day 13.

And that’s it, we have all we need.

$$public key: e,n$$
$$secret key: d,n$$

Safely publish public key $(e, n)$ and keep secret key $(d, n)$ in private.

When your friend wants to send you a message, she can use $E(x, e, n)$ to encrypt the message. You will be the only one who can use $D(y, d, n)$ to decrypt the message.

$$E(x,e,n)=x^e(modn)=y$$
$$D(y,d,n)=y^d(modn)=x$$

Why does it work?

The values of $e, d$ were specifically chosen to comply the following equation.

$$d\equiv e^{-1}(mod \varphi(n))$$
$$ed\equiv 1(mod \varphi(n))$$
$$ed = k\varphi(n)+1$$

Proper encryption and decryption is just a consequence of the stated facts.

$$y^d\equiv (x^e)^d\equiv x^{ed}(mod\varphi(n))$$
$$x^{ed}\equiv x^{k\varphi(n)+1}\equiv(x^{\varphi(n)})^k x(mod\varphi(n))$$
$$y^d\equiv (x^{\varphi(n)})^k x\equiv 1^kx \equiv x(mod\varphi(n))$$

Finding $\varphi(n)$ and $d$ is very easy if you know $p, q$. But $p, q$ or $\varphi(n)$ or $d$ are generally very, very difficult to find if you know just $e, n$. This is what makes RSA a cipher.

Are we done?

Unfortunately, not. RSA is not a [semantically secure] cipher. It’s just a type of mathematical function called trapdoor permutation. Even though the form I described can be found in many textbooks, it is totally insecure.

The explanation of RSA [in]security would require a bunch of further theory from probability, NP complexity and number theory. Let’s just say there are many attacks and focus on a few intuitive examples of what is semantically wrong.

When I use RSA to encrypt a stream of bytes, is it secure? First thing to notice is that $0^e = 0$ and $1^e = 1$. In advance there are only 256 bytes. It is easy for adversary to make a table and decrypt my messages just by a simple comparison.

Even if I use 4-byte values, it is still feasible to make the table. And 8-byte values are still not enough. Meet in the middle attack will break 20% of all my messages, but I omit the details, here.

Using RSA we can’t just safely encrypt any message. That is not a good property to have.

Another problem goes beyond theory and arises in practice.

Alice encrypts message X and the adversary fails to break the content. The problem is, if there’s any secondary action related to the message, the next time Alice sends this message, adversary compares X to previously seen messages and may be able to predict her actions accordingly.

This is a side-channel attack and shows that security issues might be even more subtle.

There are two problems that need to be fixed.

* not all plaintexts can be safely encrypted
* determinism of RSA encryption

No math tomorrow, I promise.

## algorithm

In [1]:
def secure_prime_generator():
    yield from [
        251352642263889039868309043894037481379002996715589396370854987834622532561522720403074015628816522584866374785754812790090831773387112312703220610291993961566100333483106513061700679351674883108504663868999773335993131871433147375498526830250690800432950741107471775936506033522777378528889986463928680062779,
        234601306906702217804957533486106543816960131695391266497422573355527800260716665381597389816091857137372406177664905766000014102540204528163683625043444669386812465309478832002368041295429725611772236019022712629169757194963880836723186721316763532024657471001347998077008043814690024358601642733925216784203,
    ]

In [2]:
def modinv(x, y):
    r, s = 1, 0
    while y:
        r, s = s, r - x // y * s
        x, y = y, x % y
    return r

In [3]:
def rsa_generate_keys():
    p, q = secure_prime_generator()
    n = p * q
    t = n - p - q + 1
    e = 65537
    d = modinv(e, t) % t
    assert (d * e) % t == 1
    return (e, n), (d, n)

In [4]:
def rsa(plaintext, public_key):
    return pow(plaintext, *public_key)

## keys

In [5]:
public_key, secret_key = rsa_generate_keys()

In [6]:
public_key

(65537,
 58967658369561163583995664151705537612631456941226585145001736155445085885436956133402962616775555500479429922140321605063456075222335023020218578571558003435174909963319619244821157746252197885628802071763470174413201522569356053296685834595362968800778468737693074613267684084217204017873750446802044584084498581219849973790017343888256411013653688556278788070745635045095995056877259642839730825907965544973672656542601570609068817838234644958846427643088478240335082249677864789882511592486797239674160452077169411971273434857626735582274817190984442183721945999865859466422472845277588368259261760233826535480137)

In [8]:
secret_key

(32639742054323523661031580828650534544392003478949839063736255562124081596351847364013089886417596950354636310108218358259943735367279937975211699593540109138569129405212055903155962561652878992005591100527818545966603574053221236696683939389678915058929150433015761702105657992264877747720954135956649973789334911071168428227464085150820871588160770978551544646965210798269197906675922224772713666123225990305644372957419486169245295190574189157389340237417783311258488777336686103120891002317113842264416737708675921812070527474901946450952078789439410581693777829144977217172397092723130874770379072485175449578961,
 5896765836956116358399566415170553761263145694122658514500173615544508588543695613340296261677555550047942992214032160506345607522233502302021857857155800343517490996331961924482115774625219788562880207176347017441320152256935605329668583459536296880077846873769307461326768408421720401787375044680204458408449858121984997379001734388825641101365368855627878807074563504509599505

## encryption

In [9]:
ciphertext = rsa(0xfacade17, public_key)
ciphertext

37930332066974779164345751815395228520383070918149654734779452314315201401578073995623676191126275180229634519536130536469650873769044314422167044966349923400485329143076986636753828936028839818231151694765870620828795561931693647118671097571595175457792224986081168799650305451662018565631898905521228230362451756374795462396041777695041246353088986331124068828091083876264418231632281088387993565061643784457276421962040835851861587580705505405821148340529587678730109599751672134343435130195273271395130139199246695649453457192908607858557747694289049617690930150092441884659797085102908069133974365162807747676330

In [10]:
plaintext = rsa(ciphertext, secret_key)
hex(plaintext)

'0xfacade17'